# LightGBM parameter tuning using Baysian optimization

In [1]:
from fastai import *
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import itertools
from sklearn.metrics import roc_auc_score
from skopt import gp_minimize
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [4]:
TRAIN = 'data/train.csv'
TEST = 'data/test.csv'
SAMPLE = 'data/sample_submission.csv'
train = pd.read_csv(TRAIN)
test = pd.read_csv(TEST)
X = train.drop(['ID_code', 'target'], axis=1)
y = train.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
??gp_minimize

In [ ]:
baysian_space  = [
          Integer(5, 20, name='num_leaves'),
          Integer(50, 200, name='min_child_samples'),
          Integer(20, 50,  name='min_data_in_leaf'),
          Integer(1, 5, name='bagging_freq'),
          Real(0.6, 0.9, name='subsample'),
          Real(0.01, 0.1, name='feature_fraction'),
          Real(0.001, 0.01, name='learning_rate'),
          Real(0.1, 0.5, name='bagging_fraction'),
         ]

In [ ]:
trn_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_test, label=y_test)

In [ ]:
def lgb_optim(values):
    params = {'num_leaves': values[0], 
          'min_child_samples': values[1], 
          'min_data_in_leaf': values[2], 
          'bagging_freq': values[3],
            'subsample': values[4],
            'feature_fraction': values[5],
             'learning_rate':values[6],
             'bagging_fraction': values[7],
             'boosting_type': 'gbdt',
             'objective': 'binary',
              'max_depth':-1,
              'metric':'auc',
              'boost_from_average':'false',
               'verbosity': -1,
              'objective': 'binary',
              'tree_learner': 'serial',
               'gpu_platform_id': -1,
             }
    print('\nNext set of params.....',params)
    trn_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_test, label=y_test)
    model_lgb     = lgb.train(params, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 4000)
    auc = -roc_auc_score(y_train, model_lgb.predict(X_train))
    return  auc

In [ ]:
res_gp = gp_minimize(lgb_optim, baysian_space, n_calls=20,
                     random_state=0,n_random_starts=10)

In [ ]:
clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 4000)

# RandomForest parameter tuning using Baysian optimization

In [ ]:
rf_baysian_space  = [
          Integer(15, 200, name='min_samples_leaf'),
          Integer(100, 250, name='n_estimators'),
          Real(0.1, 0.7, name='max_features'),
         ]

In [ ]:
def rf_optim(values):
    params = {'min_samples_leaf': values[0], 
          'n_estimators': values[1], 
          'max_features': values[2], 
           'n_jobs' : -1,
           'random_state' : 42,
             }
    print('\nNext set of params.....',params)
    rf = RandomForestClassifier(**params)
    rf.fit(X_train, y_train)
    auc = -roc_auc_score(y_train, rf.predict(X_train))
    return  auc

In [ ]:
res_gp = gp_minimize(rf_optim, rf_baysian_space, n_calls=10,
                     random_state=0,n_random_starts=10)

# KFold

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
def k_fold_lgb_optim(values):
    params = {'num_leaves': values[0], 
          'min_child_samples': values[1], 
          'min_data_in_leaf': values[2], 
          'bagging_freq': values[3],
            'subsample': values[4],
            'feature_fraction': values[5],
             'learning_rate':values[6],
             'bagging_fraction': values[7],
             'boosting_type': 'gbdt',
             'objective': 'binary',
              'max_depth':-1,
              'metric':'auc',
              'boost_from_average':'false',
               'verbosity': -1,
              'objective': 'binary',
              'tree_learner': 'serial',
               'gpu_platform_id': -1,
             }
    print('\nNext set of params.....',params)
    skf = StratifiedKFold(n_splits=10)
    for train_idx, test_idx in skf.split(X, y):
        trn_data = lbd.Dataset(X[train_idx], label=y[train_idx])
        test_data = lbd.Dataset(X[test_idx], label=y[test_idx])
        model_lgb     = lgb.train(params, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 4000)
        auc = roc_auc_score(y[test_idx], model_lgb.predict(X[test_idx]))
        return -auc

In [ ]:
res_gp = gp_minimize(lgb_optim, baysian_space, n_calls=10,
                     random_state=0, n_random_starts=10)

# XGBoost Tuning

In [3]:
import xgboost as xgb

In [ ]:
xg_baysian_space  = [
          Integer(50, 100, name='n_estimators'),
          Integer(10, 25, name='max_depth'),
          Integer(2, 10, name='max_delta_step'),
          Integer(2, 20, name='min_child_weight'),
          Real(0.001, 0.1, name='learning_rate'),
          Real(0.1, 0.8, name='subsample'),
         ]

In [ ]:
def xg_optim(values):
    params = {
        "metric" : 'auc',
        "n_estimators": values[0],
        "learning_rate" : values[4],
        'max_delta_step': values[2],
        'min_child_weight': values[3],
        'subsample':values[5],
        "seed": 666,
        'num_class': 4,
        'silent': False
    }
    print('\nNext set of params.....',params)
    xgbtrain = xgb.DMatrix(X_train, label=y_train)
    xgbval = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(xgbtrain, 'train'), (xgbval, 'valid')]
    model = xgb.train(params, xgbtrain, 100000, watchlist,early_stopping_rounds=4000, verbose_eval=5000)
    auc = -roc_auc_score(y_train, model.predict(xgbval))
    print(f"AUC: {auc}")
    return  auc

In [ ]:
res_gp = gp_minimize(xg_optim, xg_baysian_space, n_calls=10,
                     random_state=0,n_random_starts=10)

In [18]:
params = {
       "metric" : 'auc',
       "n_estimators": 200,
       "learning_rate" : 0.1,
       'min_child_weight': 20,
       'max_depth': 5,
       'subsample':0.7,
       "seed": 42,
       'num_class': 4,
       'silent': False
   }
print('\nNext set of params.....',params)
xgbtrain = xgb.DMatrix(X_train, label=y_train)
xgbval = xgb.DMatrix(X_test, label=y_test)
watchlist = [(xgbtrain, 'train'), (xgbval, 'valid')]
model = xgb.train(params, xgbtrain, 1000, watchlist,early_stopping_rounds=400, verbose_eval=500)


Next set of params..... {'metric': 'auc', 'n_estimators': 200, 'learning_rate': 0.1, 'min_child_weight': 20, 'max_depth': 5, 'subsample': 0.7, 'seed': 42, 'num_class': 4, 'silent': False}
[23:03:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[23:03:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[23:03:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[0]	train-merror:0.099807	valid-merror:0.10152
Multiple eval metrics have been passed: 'valid-merror' will be used for early stopping.

Will train until valid-merror hasn't improved in 400 rounds.
[23:03:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[

[23:03:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[23:03:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[23:03:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[23:03:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[23:03:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:24] /workspac

[23:03:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[23:03:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[23:03:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[23:03:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[23:03:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:41] /workspac

[23:03:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[23:03:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[23:03:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[23:03:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[23:03:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:03:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:04:00] /workspac

[23:04:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[23:04:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[23:04:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:04:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:04:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[23:04:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[23:04:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:04:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:04:17] /workspac

...

[23:18:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[23:18:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[23:18:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[23:18:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[23:18:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:05] /workspac

[23:18:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[23:18:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=5
[23:18:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[23:18:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[23:18:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:21] /workspac

[23:18:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[23:18:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[23:18:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[23:18:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=5
[23:18:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:37] /workspac

[23:18:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[23:18:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[23:18:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=5
[23:18:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[23:18:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:18:54] /workspac

[23:19:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[23:19:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[23:19:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:19:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:19:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[23:19:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[23:19:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:19:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:19:10] /workspac

In [20]:
train_auc = roc_auc_score(y_train, model.predict(xgbtrain))
test_auc = roc_auc_score(y_test, model.predict(xgbval))
print(f"Test: {test_auc}\t|\tTrain : {train_auc}")

Test: 0.6641355017446312	|	Train : 0.8783944111323778


# TODO
https://imbalanced-learn.org/en/stable/api.html